In [5]:
import numpy as np
import scipy as sp
import os
from scipy.io import wavfile
import lovely_numpy as ln
import matplotlib.pyplot as plt
import json_tricks

In [6]:
file_path = '/workspaces/linear-algebra-part-2/.laborantum/texts/Linear Algebra Part II/Discrete Fourier Transform/Audio Signal Compression/sample.wav'
try:
    # Read the WAV file
    sample_rate, data = wavfile.read(file_path)
    print(f"Sample rate: {sample_rate}, Data shape: {data.shape}")

    # Write the WAV file
    wavfile.write('original.wav', sample_rate, data)
    print("File written successfully.")

    # Uncomment if you want to process the data
    # data = data.mean(axis=1)
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Not a WAV file. RIFF form type is b'\x0b\x00WA'.


In [3]:
sample_rate, data = wavfile.read('sample.wav')
wavfile.write('original.wav', sample_rate, data)

# data = data.mean(axis=1)

ValueError: Not a WAV file. RIFF form type is b'\x0b\x00WA'.

In [ ]:
print("ORIGINAL DATA")
print(ln.lovely(data))

data = data[:, 0] + data[:, 1] * 1.j
wavfile.write('original.wav', sample_rate, np.array([data.real, data.imag]).T)

In [ ]:
def compress(input, frequencies=1000):
    a = []
    ### YOUR CODE HERE ###
    
    fft_data = np.fft.fft(input)
    magnitude = np.abs(fft_data)
    largest_indices = np.argsort(magnitude)[-frequencies:]
    compressed = [(index, fft_data[index]) for index in largest_indices]

    return np.array(compressed)

def decompress(input, data_len=191027):
    ### YOUR CODE HERE ###
    
    fft_data = np.zeros(data_len, dtype=np.complex128)
    
    for index, value in input:
        fft_data[int(index)] = value
    
    res = np.fft.ifft(fft_data)
    

    return res

In [ ]:
compressed = compress(data, frequencies=20000)

json_tricks.dump(compressed, '.answer.json')

decompressed = decompress(compressed)

print(ln.lovely(data))
print(ln.lovely(decompressed))

In [ ]:
plt.plot(data.real[:100])
plt.plot(decompressed.real[:100])

In [ ]:
wavfile.write('decompressed.wav', sample_rate, np.array([decompressed.real, decompressed.imag]).T.astype('int16'))

In [ ]:
wavfile.write('original.wav', sample_rate, np.array([data.real, data.imag]).T.astype('int16'))

In [ ]:
from IPython.display import Audio

Audio("decompressed.wav")

In [ ]:
from IPython.display import Audio

Audio("sample.wav")